# רשתות גנרטיביות

רשתות עצביות חוזרות (RNNs) והגרסאות שלהן עם תאים מבוקרים, כמו תאי זיכרון לטווח ארוך (LSTMs) ויחידות חוזרות מבוקרות (GRUs), מספקות מנגנון למידול שפה, כלומר, הן יכולות ללמוד את סדר המילים ולספק תחזיות למילה הבאה ברצף. הדבר מאפשר לנו להשתמש ב-RNNs עבור **משימות גנרטיביות**, כמו יצירת טקסט רגיל, תרגום מכונה ואפילו יצירת כיתוב לתמונות.

בארכיטקטורת RNN שדנו בה ביחידה הקודמת, כל יחידת RNN הפיקה את מצב החבוי הבא כיציאה. עם זאת, ניתן גם להוסיף יציאה נוספת לכל יחידה חוזרת, שתאפשר לנו להפיק **רצף** (ששווה באורכו לרצף המקורי). יתרה מכך, ניתן להשתמש ביחידות RNN שאינן מקבלות קלט בכל שלב, אלא רק לוקחות וקטור מצב התחלתי, ואז מפיקות רצף של יציאות.

במחברת זו, נתמקד במודלים גנרטיביים פשוטים שעוזרים לנו ליצור טקסט. לשם פשטות, נבנה **רשת ברמת תווים**, שמייצרת טקסט אות אחר אות. במהלך האימון, יש לקחת קורפוס טקסט כלשהו ולחלק אותו לרצפי אותיות.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## בניית אוצר מילים של תווים

כדי לבנות רשת גנרטיבית ברמת תווים, עלינו לפצל טקסט לתווים בודדים במקום למילים. שכבת `TextVectorization` שבה השתמשנו קודם אינה יכולה לעשות זאת, ולכן יש לנו שתי אפשרויות:

* לטעון טקסט באופן ידני ולבצע טוקניזציה 'ביד', כפי שמוצג [בדוגמה הרשמית הזו של Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* להשתמש במחלקת `Tokenizer` לטוקניזציה ברמת תווים.

נבחר באפשרות השנייה. ניתן להשתמש ב-`Tokenizer` גם לטוקניזציה למילים, כך שניתן לעבור בקלות מטוקניזציה ברמת תווים לטוקניזציה ברמת מילים.

כדי לבצע טוקניזציה ברמת תווים, יש להעביר את הפרמטר `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

אנחנו גם רוצים להשתמש בטוקן מיוחד אחד כדי לציין **סוף רצף**, אותו נקרא `<eos>`. בואו נוסיף אותו ידנית לאוצר המילים:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

כעת, כדי לקודד טקסט לרצפי מספרים, אנו יכולים להשתמש:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## אימון RNN גנרטיבי ליצירת כותרות

הדרך שבה נאמן RNN לייצר כותרות חדשות היא כדלקמן. בכל שלב, ניקח כותרת אחת, שתוזן לתוך RNN, ולכל תו קלט נבקש מהרשת לייצר את תו הפלט הבא:

![תמונה המציגה דוגמה ליצירת המילה 'HELLO' באמצעות RNN.](../../../../../translated_images/he/rnn-generate.56c54afb52f9781d.webp)

עבור התו האחרון ברצף שלנו, נבקש מהרשת לייצר את הטוקן `<eos>`.

ההבדל העיקרי בין RNN גנרטיבי שאנו משתמשים בו כאן הוא שניקח פלט מכל שלב של ה-RNN, ולא רק מהתא האחרון. ניתן להשיג זאת על ידי הגדרת הפרמטר `return_sequences` לתא ה-RNN.

לכן, במהלך האימון, הקלט לרשת יהיה רצף של תווים מקודדים באורך מסוים, והפלט יהיה רצף באותו אורך, אך מוזז באלמנט אחד ומסתיים ב-`<eos>`. המיני-באטץ' יכיל כמה רצפים כאלה, ונצטרך להשתמש **בריפוד** כדי ליישר את כל הרצפים.

בואו ניצור פונקציות שיתאימו את מערך הנתונים עבורנו. מכיוון שאנחנו רוצים לרפד רצפים ברמת המיני-באטץ', קודם נבצע באטץ' למערך הנתונים על ידי קריאה ל-`.batch()`, ואז נשתמש ב-`map` כדי לבצע את ההתאמה. כך, פונקציית ההתאמה תקבל מיני-באטץ' שלם כפרמטר:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

כמה דברים חשובים שאנחנו עושים כאן:
* קודם כל אנחנו שולפים את הטקסט עצמו מתוך ה-String Tensor
* `text_to_sequences` ממיר את רשימת המחרוזות לרשימה של טנסורים של מספרים שלמים
* `pad_sequences` משלים את הטנסורים לאורך המקסימלי שלהם
* לבסוף, אנחנו מבצעים קידוד one-hot לכל התווים, וגם מבצעים את ההסטה והוספת `<eos>`. בקרוב נבין למה אנחנו צריכים תווים מקודדים ב-one-hot

עם זאת, הפונקציה הזו היא **Pythonic**, כלומר היא לא יכולה להיות מתורגמת אוטומטית לגרף חישובי של Tensorflow. נקבל שגיאות אם ננסה להשתמש בפונקציה הזו ישירות בתוך הפונקציה `Dataset.map`. עלינו לעטוף את הקריאה ה-Pythonic הזו באמצעות ה-wrapper `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: ההבחנה בין פונקציות טרנספורמציה בסגנון פייתון לבין פונקציות טרנספורמציה של Tensorflow עשויה להיראות מורכבת מדי, ואתם אולי שואלים מדוע לא לבצע את טרנספורמציית הנתונים באמצעות פונקציות פייתון סטנדרטיות לפני שמעבירים אותם ל-`fit`. למרות שזה בהחלט אפשרי, השימוש ב-`Dataset.map` מציע יתרון משמעותי, מכיוון שצינור טרנספורמציית הנתונים מתבצע באמצעות גרף החישוב של Tensorflow, שמנצל את יכולות החישוב של ה-GPU וממזער את הצורך להעביר נתונים בין CPU ל-GPU.

עכשיו אנחנו יכולים לבנות את רשת הגנרטור שלנו ולהתחיל באימון. היא יכולה להתבסס על כל תא חוזר שדנו בו ביחידה הקודמת (פשוט, LSTM או GRU). בדוגמה שלנו נשתמש ב-LSTM.

מכיוון שהרשת מקבלת תווים כקלט וגודל אוצר המילים די קטן, אין צורך בשכבת הטמעה; קלט מקודד בשיטת one-hot יכול להיכנס ישירות לתא LSTM. שכבת הפלט תהיה מסווג `Dense` שתמיר את הפלט של LSTM למספרי טוקנים מקודדים בשיטת one-hot.

בנוסף, מכיוון שאנחנו מתמודדים עם רצפים באורך משתנה, נוכל להשתמש בשכבת `Masking` כדי ליצור מסכה שתתעלם מהחלק המרופד של המחרוזת. זה לא הכרחי לחלוטין, מכיוון שאנחנו לא מאוד מתעניינים בכל מה שמעבר לטוקן `<eos>`, אבל נשתמש בזה כדי לצבור ניסיון עם סוג השכבה הזה. ה-`input_shape` יהיה `(None, vocab_size)`, כאשר `None` מציין רצף באורך משתנה, וצורת הפלט תהיה גם `(None, vocab_size)`, כפי שניתן לראות מה-`summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## יצירת פלט

עכשיו, לאחר שאימנו את המודל, אנחנו רוצים להשתמש בו כדי ליצור פלט. קודם כל, אנחנו צריכים דרך לפענח טקסט שמיוצג כרצף של מספרי טוקנים. כדי לעשות זאת, נוכל להשתמש בפונקציה `tokenizer.sequences_to_texts`; עם זאת, היא לא עובדת טוב עם טוקניזציה ברמת תווים. לכן ניקח מילון של טוקנים מהטוקנייזר (שנקרא `word_index`), נבנה מפת היפוך, ונכתוב פונקציית פענוח משלנו:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

עכשיו, בואו נתחיל ביצירה. נתחיל עם מחרוזת `start`, נקודד אותה לרצף `inp`, ואז בכל שלב נקרא לרשת שלנו כדי להסיק את התו הבא.

הפלט של הרשת `out` הוא וקטור עם `vocab_size` אלמנטים שמייצגים את ההסתברויות של כל טוקן, ואנחנו יכולים למצוא את מספר הטוקן הסביר ביותר באמצעות `argmax`. לאחר מכן נוסיף את התו הזה לרשימת הטוקנים שנוצרו, ונמשיך בתהליך היצירה. התהליך הזה של יצירת תו אחד חוזר על עצמו `size` פעמים כדי ליצור את מספר התווים הנדרש, ואנחנו מסיימים מוקדם אם נתקלנו ב-`eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## דגימת פלט במהלך האימון

מכיוון שאין לנו מדדים שימושיים כמו *דיוק*, הדרך היחידה שבה נוכל לראות שהמודל שלנו משתפר היא על ידי **דגימה** של מחרוזות שנוצרות במהלך האימון. כדי לעשות זאת, נשתמש ב**callbacks**, כלומר פונקציות שנוכל להעביר לפונקציה `fit`, ושיקראו באופן תקופתי במהלך האימון.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


דוגמה זו כבר מייצרת טקסט די טוב, אך ניתן לשפר אותה בכמה דרכים:

* **יותר טקסט**. השתמשנו רק בכותרות עבור המשימה שלנו, אך ייתכן שתרצה להתנסות בטקסט מלא. זכור ש-RNNs אינם מצטיינים בהתמודדות עם רצפים ארוכים, ולכן הגיוני או לחלק אותם למשפטים קצרים יותר, או תמיד להתאמן על רצף קבוע באורך מוגדר מראש `num_chars` (לדוגמה, 256). תוכל לנסות לשנות את הדוגמה לעיל לארכיטקטורה כזו, תוך שימוש ב[מדריך הרשמי של Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) כהשראה.

* **LSTM רב-שכבתי**. הגיוני לנסות 2 או 3 שכבות של תאי LSTM. כפי שציינו ביחידה הקודמת, כל שכבה של LSTM מוציאה דפוסים מסוימים מהטקסט, ובמקרה של מחולל ברמת תווים, ניתן לצפות שהשכבה הנמוכה של LSTM תהיה אחראית על חילוץ הברות, ושכבות גבוהות יותר - על מילים ושילובי מילים. ניתן ליישם זאת בפשטות על ידי העברת פרמטר מספר-שכבות לבנאי של LSTM.

* ייתכן שתרצה גם להתנסות עם **יחידות GRU** ולראות אילו מהן מבצעות טוב יותר, וכן עם **גדלים שונים של שכבות נסתרות**. שכבה נסתרת גדולה מדי עשויה לגרום לבעיה של התאמת יתר (למשל, הרשת תלמד את הטקסט המדויק), וגודל קטן יותר עשוי שלא להפיק תוצאה טובה.


## יצירת טקסט רך וטמפרטורה

בהגדרה הקודמת של `generate`, תמיד בחרנו את התו עם ההסתברות הגבוהה ביותר כתו הבא בטקסט שנוצר. הדבר הוביל לכך שהטקסט לעיתים קרובות "חזר" על רצפי תווים זהים שוב ושוב, כמו בדוגמה הזו:
```
today of the second the company and a second the company ...
```

עם זאת, אם נבחן את התפלגות ההסתברויות עבור התו הבא, ייתכן שההבדל בין כמה מההסתברויות הגבוהות ביותר אינו גדול, לדוגמה: תו אחד יכול להיות בעל הסתברות של 0.2, ותו אחר - 0.19, וכו'. לדוגמה, כאשר מחפשים את התו הבא ברצף '*play*', התו הבא יכול להיות באותה מידה רווח או **e** (כמו במילה *player*).

מסקנה זו מובילה אותנו להבנה שלא תמיד "הוגן" לבחור את התו עם ההסתברות הגבוהה ביותר, משום שבחירה בתו עם ההסתברות השנייה הגבוהה עדיין יכולה להוביל לטקסט משמעותי. חכם יותר **לדגום** תווים מתוך התפלגות ההסתברויות שניתנת על ידי פלט הרשת.

דגימה זו יכולה להתבצע באמצעות הפונקציה `np.multinomial`, שמיישמת את מה שנקרא **התפלגות מולטינומית**. פונקציה שמיישמת יצירת טקסט **רך** מוגדרת להלן:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

הוספנו פרמטר נוסף שנקרא **טמפרטורה**, אשר משמש לציון עד כמה עלינו להיצמד להסתברות הגבוהה ביותר. אם הטמפרטורה היא 1.0, אנו מבצעים דגימה מולטינומית הוגנת, וכאשר הטמפרטורה עולה לאינסוף - כל ההסתברויות הופכות לשוות, ואנו בוחרים באופן אקראי את התו הבא. בדוגמה למטה ניתן לראות שהטקסט הופך לחסר משמעות כאשר אנו מעלים את הטמפרטורה יותר מדי, והוא מזכיר טקסט "מחזורי" שנוצר באופן קשיח כאשר הטמפרטורה מתקרבת ל-0.



---

**כתב ויתור**:  
מסמך זה תורגם באמצעות שירות תרגום מבוסס בינה מלאכותית [Co-op Translator](https://github.com/Azure/co-op-translator). בעוד שאנו שואפים לדיוק, יש להיות מודעים לכך שתרגומים אוטומטיים עשויים להכיל שגיאות או אי דיוקים. המסמך המקורי בשפתו המקורית צריך להיחשב כמקור סמכותי. עבור מידע קריטי, מומלץ להשתמש בתרגום מקצועי על ידי אדם. איננו נושאים באחריות לאי הבנות או לפרשנויות שגויות הנובעות משימוש בתרגום זה.
